In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numericalunits as nu
from scipy.signal import find_peaks
from spectral_density_fit import spectral_density_fitter

In [ ]:
plt.style.use("https://johannesfeist.eu/misc/jf_cb.mplstyle")

In [ ]:
"input should be in numericalunits units"
Jωprefac = lambda ω, μ: ω**2 * μ**2 / (np.pi * nu.hbar * nu.eps0 * nu.c0**2)

"input should be in numericalunits units"


def free_space_Jω(ħω, μ):
    ω = ħω / nu.ħ
    GF_imag = ω / (6 * np.pi * nu.c0)
    pref = Jωprefac(ω, μ)
    return pref * GF_imag

# Load spectral density
Note that we use a numerically calculated Purcell factor $P(\omega)$ and calculate $J(\omega) = J_0(\omega) P(\omega)$. We here use a dipole moment of 1 e nm. The spectral densities are in "numericalunits units" of frequency, to get energy we need to multiply by $\hbar$, and then we convert to eV.

In [ ]:
ws, P = np.loadtxt("Purcell_gap.dat", unpack=True)
dw = ws[1] - ws[0]
J0 = free_space_Jω(ws * nu.eV, nu.e * nu.nm)
Jw = nu.ħ * J0 * P / nu.eV
plt.plot(ws, Jw)
plt.xlabel("ω (eV)")
plt.ylabel("J(ω) (eV)");

# Use find_peaks from scipy.signal to get starting guesses
The starting guess for $J(\omega)$ is a sum of Lorentzians, get their properties from the `find_peaks` function
The normalized Lorentzians in the spectral density have the form $L(\omega) = \frac{g_n^2}{\pi} \frac{\kappa_n/2}{(\omega-\omega_n)^2 + \kappa_n^2/4}$, and the values at the peak positions $J_n = J(\omega_n)$ are thus related to $g_n$ as $J_n = L(\omega_n) = \frac{2 g_n^2}{\pi \kappa_n}$

In [ ]:
# find "most" peaks (with some minimum width and prominence
peak_inds, peak_props = find_peaks(Jw, width=3, prominence=5e-4)

# number of modes
Nm = len(peak_inds)
# we have 1 emitter
Ne = 1

print(f"we use {Nm} modes in the fit")

# make fitter object (is actually an nlopt object, with some functions added by spectral_density_fit)
# we set fitlog=True to fit the logarithm of the spectral density, not the spectral density itself
# (this makes the fit more sensitive to small values)
opt = spectral_density_fitter(ws, Jw, Nm, fitlog=True)

# start fit with a diagonal Hamiltonian, with parameters taken from those found by find_peaks
H = np.diag(ws[peak_inds])
kappas = dw * peak_props["widths"]
gs = np.sqrt(np.pi * kappas * Jw[peak_inds] / 2).reshape(1, Nm)  # g has to be Ne x Nm array

# "ps" is the 1d array of fit parameters, obtain it from initial guesses for H,κ,g
ps = opt.Hκg_to_ps(H, kappas, gs)
# evaluate the model spectral density with those parameters
# .squeeze() because Jfun returns [Ne,Ne,Nω] array also for Ne=1, squeeze removes dimensions of size 1
# (i.e., transforms Jmod.shape from [1,1,len(ω)] -> [len(ω)])
Jmod = opt.Jfun(ws, ps).squeeze()

plt.plot(ws, Jw, label="numerical")
plt.plot(ws[peak_inds], Jw[peak_inds], "o", label="identified peaks")
plt.plot(ws, Jmod, label="guessed model")
plt.xlabel("ω (eV)")
plt.ylabel("J(ω) (eV)")
plt.yscale("log")
plt.legend();

# do the optimization
Note that for this example, we did not actually use enough modes and so the fit is not great. Usually one would take the guesses from `find_peaks` and then refine them manually (i.e., already include more than one mode for the broad non-Lorentzian pseudomode peak at 2.5 eV, etc).

In [ ]:
# set the optimization tolerance
opt.set_ftol_rel(1e-6)
ps = opt.optimize(ps)
Jmod = opt.Jfun(ws, ps).squeeze()

In [ ]:
plt.plot(ws, Jw, label="numerical")
plt.plot(ws, Jmod, label="fitted model")
plt.xlabel("ω (eV)")
plt.ylabel("J(ω) (eV)")
plt.yscale("log")
plt.legend();

# Implement few-mode model

We want to treat the system with $N_m$ cavity modes and $N_e$ emitters with Hamiltonian (within RWA)
\begin{equation}
H = \sum_{ij} \omega_{ij} a_i^\dagger a_j + \sum_\alpha \omega_{e,\alpha} \sigma_\alpha^\dagger \sigma_\alpha + \sum_{\alpha,i} g_{\alpha i} (\sigma_\alpha^\dagger a_i + \sigma_\alpha a_i^\dagger)
\end{equation}
and Lindblad decay terms $\kappa_i \mathcal{L}_{a_i}[\rho]$.

Note that since we didn't use the correct units for $J$ above, our $g$'s also have the wrong ones. So here at the latest we should multiply by the square root of the correct prefactor. We invent a number here for the example...

In [ ]:
from qutip import *

In [ ]:
# spectral_density_fitter uses jax, convert to normal numpy arrays for further use
H, kappas, gs = map(np.array, opt.ps_to_Hκg(ps))
display(Qobj(H))
display(Qobj(kappas.reshape(1, -1)))
display(Qobj(gs))

In [ ]:
# emitter frequency
we = 1.2

In [ ]:
# maximum number of excitations
Nexc = 1
# dimensions of the quantum operators.
# Since the only restriction we want is in the total number of excitations,
# we allow each individual photonic mode to have up to Nexc+1 states (i.e., 0 to Nexc photons),
# while the emitters are two-level systems
part_dims = [Nexc + 1] * Nm + [2] * Ne
print("Dimensions of quantum operators:", part_dims)
print("Full Hilbert space would have size", np.prod(part_dims))

# this creates a list of annihilation operators for subsystems with dimensions given by part_dims,
# but only allowing up to Nexc excitations in the system
ann_ops = enr_destroy(part_dims, Nexc)
# the first Nm operators are the photon mode operators a_i
aops = ann_ops[:Nm]
# the rest are the Ne emitter operators σ_α
σs = ann_ops[Nm:]
assert len(σs) == Ne
print("excitation-number restricted Hilbert space for up to", Nexc, "excitations has size", σs[0].shape[0])

In [ ]:
# since A and Ad are arrays, we can use the matrix multiplication operator
H = sum(H[i, j] * aops[i].dag() * aops[j] for i in range(Nm) for j in range(Nm)) + sum(we * σs[i].dag() * σs[i] for i in range(Ne))
# very important: always use normal ordering in excitation-number restricted subspace (i.e., creation operators on the left)
H += sum(gs[i, j] * (aops[j].dag() * σs[i] + σs[i].dag() * aops[j]) for i in range(Ne) for j in range(Nm))
# annihilation operators
c_ops = [np.sqrt(ka) * a for (ka, a) in zip(kappas, aops)]
# calculate, e.g., expected populations of all subsystems
e_ops = [x.dag() * x for x in ann_ops]

In [ ]:
# start with the emitter excited
ψ0 = enr_fock(part_dims, Nexc, np.r_[np.zeros(Nm), 1])

# tsfs is time in femtoseconds, for QuTiP we should use time units of ħ/eV (since ħ=1 and our energy units are eV)
tsfs = np.linspace(0, 50, 201)
# tsfs*nu.fs converts to "internal" numericalunits time unit, dividing by nu.ħ/nu.eV converts that to ħ/eV
ts = tsfs * nu.fs / (nu.ħ / nu.eV)

In [ ]:
sol = mesolve(H, ψ0, ts, c_ops, e_ops=e_ops)

In [ ]:
plt.plot(tsfs, sol.expect[-1], label="emitter")
for ii in range(Nm):
    plt.plot(tsfs, sol.expect[ii], label=f"photon mode {ii + 1}")
plt.xlabel("t (fs)")
plt.ylabel("population")
plt.legend(ncol=2);